In [1]:
import numpy as np
import open3d
import copy
import tqdm

In [2]:
def read_features_file(file_path):
    data = np.load(file_path)
    features = open3d.registration.Feature()
    features.data = data["features"].T
    keypts = open3d.PointCloud()
    keypts.points = open3d.Vector3dVector(data["keypts"])
    scores = data["scores"]
    return keypts, features, scores


def execute_global_registration(src_keypts, tgt_keypts, src_desc, tgt_desc, distance_threshold):
    result = open3d.registration_ransac_based_on_feature_matching(
        src_keypts, tgt_keypts, src_desc, tgt_desc,
        distance_threshold,
        open3d.TransformationEstimationPointToPoint(False), 3,
        [open3d.CorrespondenceCheckerBasedOnEdgeLength(0.9),
         open3d.CorrespondenceCheckerBasedOnDistance(distance_threshold)],
        open3d.RANSACConvergenceCriteria(4000000, 500))
    return result


In [3]:
transformations = np.load("data/GroundTruthTransforms/ground_truth_transformations.npz")

In [4]:
source_id = transformations["source_ids"][0]
target_id = transformations["target_ids"][0]
is_match = transformations["correct_matches"][0]
transformation_gt = transformations["transformations"][0]

src_features_file = f"data/Features/0.03/{source_id}.npz"
tgt_features_file = f"data/Features/0.03/{target_id}.npz"

src_keypts, src_features, src_scores = read_features_file(src_features_file)
tgt_keypts, tgt_features, tgt_scores = read_features_file(tgt_features_file)

# src_keypts.transform(result_ransac.transformation)

# open3d.draw_geometries([src_keypts, tgt_keypts])

In [5]:
# source = src_keypts
# target = tgt_keypts
#
# source_features = src_features
# target_features = tgt_features
#
# max_correspondence_distance = 0.05
# estimation = open3d.TransformationEstimationPointToPoint(False)
#
# ransac_n = 3
#
# checkers = [open3d.CorrespondenceCheckerBasedOnEdgeLength(0.9), open3d.CorrespondenceCheckerBasedOnDistance(max_correspondence_distance)]
#
# criteria = open3d.RANSACConvergenceCriteria(4000000, 500)

In [12]:
def ransac_feature_matching(source, target, source_features, target_features, max_correspondence_distance, estimation, ransac_n, checkers, criteria):
    total_validation = 0
    finished_validation = False
    similar_features = [[] for i in range(len(source.points))]

    registration_result = None

    kdtree_feature = open3d.geometry.KDTreeFlann(target_features)

    limit_max = np.asarray(source.points).max(axis=0)
    limit_min = np.asarray(source.points).min(axis=0)

    count = 0

    for i in tqdm.trange(criteria.max_iteration):
        if not finished_validation:
            ransac_corres = []
            for j in range(ransac_n):
                source_sample_id = np.random.randint(0, len(source.points))
                if len(similar_features[source_sample_id]) == 0:
                    _, indices, dists = kdtree_feature.search_knn_vector_xd(source_features.data[:, source_sample_id], 1)
                    similar_features[source_sample_id] = list(indices)
                ransac_corres.append([source_sample_id, similar_features[source_sample_id][0]])
            check = True

            ransac_corres = open3d.utility.Vector2iVector(ransac_corres)
            transformation = estimation.compute_transformation(source, target, ransac_corres)

            for checker in checkers:
                if not checker.Check(source, target, ransac_corres, transformation):
                    check = False

            if not check: continue

            pcd = copy.deepcopy(source)
            pcd.transform(transformation)

            result = open3d.registration.evaluate_registration(pcd, target, max_correspondence_distance, transformation)

            if np.all(result.transformation[:3, 3] > limit_max):
                count += 1
                continue

            if np.all(result.transformation[:3, 3] < limit_min):
                count += 1
                continue

            if not registration_result or registration_result.fitness < result.fitness or registration_result.inlier_rmse > result.inlier_rmse:
                    registration_result = result

            total_validation += 1
            if total_validation >= criteria.max_validation:
                finished_validation = True
    print(count)
    return registration_result

In [7]:
result_ransac = ransac_feature_matching(
    src_keypts, tgt_keypts, src_features, tgt_features,
    0.05, open3d.TransformationEstimationPointToPoint(False), 3,
    [open3d.CorrespondenceCheckerBasedOnEdgeLength(0.9), open3d.CorrespondenceCheckerBasedOnDistance(0.05)],
    open3d.RANSACConvergenceCriteria(4000000, 500)
)

100%|██████████| 4000000/4000000 [00:05<00:00, 786007.49it/s] 


In [8]:
src_keypts.transform(result_ransac.transformation)

open3d.draw_geometries([src_keypts, tgt_keypts])

In [13]:
for i in np.arange(transformations["source_ids"].shape[0]):
    source_id = transformations["source_ids"][i]
    target_id = transformations["target_ids"][i]
    is_match = transformations["correct_matches"][i]
    transformation = transformations["transformations"][i]
    path = transformations["paths"][i]

    src_features_file = f"data/Features/0.03/{source_id}.npz"
    tgt_features_file = f"data/Features/0.03/{target_id}.npz"

    src_keypts, src_features, src_scores = read_features_file(src_features_file)
    tgt_keypts, tgt_features, tgt_scores = read_features_file(tgt_features_file)

    result_ransac = ransac_feature_matching(
        src_keypts, tgt_keypts, src_features, tgt_features,
        0.05, open3d.TransformationEstimationPointToPoint(False), 3,
        [open3d.CorrespondenceCheckerBasedOnEdgeLength(0.9), open3d.CorrespondenceCheckerBasedOnDistance(0.05)],
        open3d.RANSACConvergenceCriteria(4000000, 500)
    )

    # src_keypts.transform(result_ransac.transformation)

    # open3d.draw_geometries([src_keypts, tgt_keypts])

100%|██████████| 4000000/4000000 [00:05<00:00, 786339.75it/s] 


0


100%|██████████| 4000000/4000000 [00:06<00:00, 649844.12it/s] 


0


100%|██████████| 4000000/4000000 [00:34<00:00, 117144.97it/s] 


...

0


100%|██████████| 4000000/4000000 [00:15<00:00, 257217.94it/s] 


0


100%|██████████| 4000000/4000000 [00:56<00:00, 71317.81it/s]  

0
